In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install yaml

ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml


# Import local modules

In [3]:
import sys
sys.path.insert(0, '../src')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from utils.exp_saver import ExpSaver

from data_loading.load_cas import load_cas
from data_loading.load_embeddings import load_vectors
from data_loading.load_neg_filters import load_filters

from data_processing.preprocessing import preprocess_data, add_tags_per_word

from model.LSTM import embed_bilstm

from train.train import CV_train, format_results

ModuleNotFoundError: No module named 'yaml'

# Import universal modules

In [4]:
import pandas as pd
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',None)

import numpy as np
import glob

# Parameters

In [5]:
params = {
    'pretrainedEmbed': 'bin', # True or False
    'neg_only': 'filter', # False, True or 'filter'
    'n_fold': 10,
    'batch_size': 512,
    'epochs': 30,
    
    # nn hyper parameters
    'hyper_params': {
        'lr': 1e-3,
        'lr_decay': 1e-6,
    },
    
    # nn structure related
    'model_structure': {
        'with_crf': False,
        'word_embed': 300,
        'pos_embed': 10,
        'cue_embed': 10,
        'lstm_size': 200,
    },
}

# Load corpus

In [6]:
cas = load_cas(remove_punctuation=True)

from ipywidgets import IntProgress

# get ids of sentences containing negation
negative_sentence_ids = cas.groupby('sentence_id').apply(lambda x : x['label'].str.contains('neg')).dropna().index.get_level_values(0).unique()

classes = ['O_neg', 'B_neg', 'I_neg']

  1%|          | 566/75258 [00:00<00:13, 5556.14it/s]

Grouping negation


100%|██████████| 75258/75258 [00:31<00:00, 2363.75it/s]


In [7]:
# CONVERT TO BIO NCRFpp format

# cas_bio = load_cas(remove_punctuation=False)[['sentence_id', 'word_id', 'word', 'label']]

# cas_spaces = cas_bio.groupby('sentence_id').apply(
#     lambda d: d.append({'sentence_id': d.name}, ignore_index=True).astype({'sentence_id': int})
# ).reset_index(drop=True)

# cas_spaces[['word', 'label']].to_csv('../data/corpus_dalloux/cas_BIO.csv',  header=False, index=False, sep='\t')

# Retain only neg or not

In [8]:
neg_only = load_filters() if params['neg_only']=='filter' else params['neg_only']

if isinstance(neg_only, str):
    df1 = add_tags_per_word(cas, neg_only)  
    merge = cas.reset_index().merge(df1, left_index=True, right_index=True)
    neg_sentences = merge[merge['label_y'].str.contains('B_neg|I_neg')]['sentence_id_x'].unique()
    cas_neg = cas[cas['sentence_id'].isin(neg_sentences)]
elif neg_only==True:
    cas_neg = cas.groupby('sentence_id').filter(lambda x: (x['label'].str.contains('B').any()) or (x['label'].str.contains('I').any()))
else:
    cas_neg = cas.copy()
        
# if neg_only:
#     reset_sentence_dic = {w: i for i,w in enumerate(cas_neg['sentence_id'].unique())}
#     cas_neg['sentence_id'] = cas_neg['sentence_id'].map(reset_sentence_dic).astype(int)

# Check filter accuracy, sentence wise

In [10]:
if not isinstance(neg_only, bool):
    from sklearn.metrics import classification_report

    true_neg = []
    pred_neg = []

    for sentence_id, sentence in cas.groupby('sentence_id'):

        true_labels = sentence['label'].unique()

        true_neg.append(('B_scope_neg' in true_labels) or ('I_scope_neg' in true_labels))

        pred_neg.append(sentence_id in neg_sentences)

        if not (sentence_id in neg_sentences) and (('B_scope_neg' in true_labels) or ('I_scope_neg' in true_labels)):
            display(sentence)


    print("Predicted support: \nO_neg: %d\nI_neg: %d"%(len(pred_neg)-sum(pred_neg), sum(pred_neg)))
    print(classification_report(true_neg, pred_neg, target_names=['O_neg', 'I_neg']))

,sentence_id,word_id,word,lem,postag,cue_tag,label
14246,652,0,Les,le,DET:ART,_,B_scope_neg
14247,652,1,résultats,résultat,NOM,_,I_scope_neg
14248,652,2,de,de,PRP,_,I_scope_neg
14249,652,3,l',le,DET:ART,_,I_scope_neg
14250,652,4,examen,examen,NOM,_,I_scope_neg
14251,652,5,histologique,histologique,ADJ,_,I_scope_neg
14252,652,6,de,de,PRP,_,I_scope_neg
14253,652,7,l',le,DET:ART,_,I_scope_neg
14254,652,8,ovariectomie,ovariectomie,NOM,_,I_scope_neg
14255,652,9,droite,droit,ADJ,_,I_scope_neg


,sentence_id,word_id,word,lem,postag,cue_tag,label
23715,1060,0,Après,après,PRP,_,_
23716,1060,1,sphinctérotomie,sphinctérotomie,NOM,_,_
23717,1060,2,de,de,PRP,_,_
23718,1060,3,la,le,DET:ART,_,_
23719,1060,4,papille,papille,NOM,_,_
23720,1060,5,principale,principal,ADJ,_,_
23722,1060,7,on,on,PRO:PER,_,_
23723,1060,8,posait,poser,VER:impf,_,_
23724,1060,9,une,un,DET:ART,_,_
23725,1060,10,prothèse,prothèse,NOM,_,_


,sentence_id,word_id,word,lem,postag,cue_tag,label
28143,1240,0,Explorations,exploration,NOM,_,_
28145,1240,2,IRM,IRM,NOM,_,_
28146,1240,3,cérébrale,cérébral,ADJ,_,_
28147,1240,4,normale,normal,ADJ,_,_
28149,1240,6,PL,PL,NOM,_,_
28150,1240,7,protéinorachie,protéinorachie,NOM,_,_
28151,1240,8,à,à,PRP,_,_
28152,1240,9,0,card,NUM,_,_
28154,1240,11,46,card,NUM,_,_
28155,1240,12,g,g,NOM,_,_


,sentence_id,word_id,word,lem,postag,cue_tag,label
29762,1319,0,HISTOIRE,histoire,NOM,_,I_scope_neg
29763,1319,1,DU,DU,NAM,_,I_scope_neg
29764,1319,2,PATIENT,PATIENT,NAM,_,I_scope_neg


,sentence_id,word_id,word,lem,postag,cue_tag,label
29881,1324,0,Mme,madame,NOM,_,_
29883,1324,2,n',ne,ADV,B_cue_neg,B_scope_neg
29884,1324,3,a,avoir,VER:pres,_,I_scope_neg
29885,1324,4,jamais,jamais,ADV,I_cue_neg,I_scope_neg
29886,1324,5,fumé,fumer,VER:pper,_,I_scope_neg
29888,1324,7,a,avoir,VER:pres,_,_
29889,1324,8,une,un,DET:ART,_,_
29890,1324,9,tension,tension,NOM,_,_
29891,1324,10,artérielle,artériel,ADJ,_,_
29892,1324,11,mesurée,mesurer,VER:pper,_,_


...

,sentence_id,word_id,word,lem,postag,cue_tag,label
78206,3425,0,Il,il,PRO:PER,_,_
78207,3425,1,n',ne,ADV,B_cue_neg,B_scope_neg
78208,3425,2,existait,exister,VER:impf,_,I_scope_neg
78209,3425,3,plus,plus,ADV,I_cue_neg,I_scope_neg
78210,3425,4,de,de,PRP,_,I_scope_neg
78211,3425,5,foyer,foyer,NOM,_,I_scope_neg
78212,3425,6,de,de,PRP,_,I_scope_neg
78213,3425,7,nécrose,nécrose,NOM,_,I_scope_neg
78214,3425,8,hépatocytaire,hépatocytaire,ADJ,_,I_scope_neg


,sentence_id,word_id,word,lem,postag,cue_tag,label
79975,3489,0,Les,le,DET:ART,_,_
79976,3489,1,cellules,cellule,NOM,_,_
79977,3489,2,tumorales,tumoral,ADJ,_,_
79978,3489,3,étaient,être,VER:impf,_,_
79979,3489,4,en,en,PRP,_,_
79980,3489,5,revanche,revanche,NOM,_,_
79981,3489,6,négatives,négatif,ADJ,B_cue_neg,B_scope_neg
79982,3489,7,aux,au,PRP:det,_,I_scope_neg
79983,3489,8,marqueurs,marqueur,NOM,_,I_scope_neg
79984,3489,9,épithéliaux,épithélial,ADJ,_,I_scope_neg


,sentence_id,word_id,word,lem,postag,cue_tag,label
82972,3602,0,Il,il,PRO:PER,_,_
82973,3602,1,ne,ne,ADV,B_cue_neg,B_scope_neg
82974,3602,2,fume,fumer,VER:pres,_,I_scope_neg
82975,3602,3,plus,plus,ADV,I_cue_neg,I_scope_neg
82976,3602,4,et,et,KON,_,_
82977,3602,5,ne,ne,ADV,B_cue_neg,B_scope_neg
82978,3602,6,boit,boire,VER:pres,_,I_scope_neg
82979,3602,7,plus,plus,ADV,I_cue_neg,I_scope_neg
82980,3602,8,d',de,PRP,_,I_scope_neg
82981,3602,9,alcool,alcool,NOM,_,I_scope_neg


,sentence_id,word_id,word,lem,postag,cue_tag,label
84503,3662,0,Traitement,traitement,NOM,_,_
84504,3662,1,habituel,habituel,ADJ,_,_
84506,3662,3,lisinopril,lisinopril,NOM,_,_
84508,3662,5,hypertension,hypertension,NOM,_,_
84509,3662,6,artérielle,artériel,ADJ,_,_
84512,3662,9,metformine,metformine,NOM,_,_
84514,3662,11,diabète,diabète,NOM,_,_
84515,3662,12,de,de,PRP,_,_
84516,3662,13,type,type,NOM,_,_
84517,3662,14,2,card,NUM,_,_


Predicted support: 
O_neg: 2888
I_neg: 902
              precision    recall  f1-score   support

       O_neg       0.99      0.96      0.98      2984
       I_neg       0.87      0.97      0.92       806

    accuracy                           0.96      3790
   macro avg       0.93      0.97      0.95      3790
weighted avg       0.97      0.96      0.96      3790



In [10]:
# WITHOUT PUNCTUATION (with punctuation, results crumble)

# Predicted support: 
# O_neg: 2888
# I_neg: 902
#               precision    recall  f1-score   support

#        O_neg       0.99      0.96      0.98      2984
#        I_neg       0.87      0.97      0.92       806

#     accuracy                           0.96      3790
#    macro avg       0.93      0.97      0.95      3790
# weighted avg       0.97      0.96      0.96      3790

# Sample one random negative sentence to check

In [11]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

random_neg_sentence = np.random.choice(negative_sentence_ids)
display(cas[cas['sentence_id']==random_neg_sentence])

pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',None)

,sentence_id,word_id,word,lem,postag,cue_tag,label
34867,1558,0,Une,un,DET:ART,_,_
34868,1558,1,semaine,semaine,NOM,_,_
34869,1558,2,plus,plus,ADV,_,_
34870,1558,3,tard,tard,ADV,_,_
34872,1558,5,et,et,KON,_,_
34873,1558,6,suite,suite,NOM,_,_
34874,1558,7,à,à,PRP,_,_
34875,1558,8,l',le,DET:ART,_,_
34876,1558,9,acupuncture,acupuncture,NOM,_,_
34878,1558,11,car,car,KON,_,_


# Check fast text vs vocab

In [ ]:
if params['pretrainedEmbed']==True:
    ft_vec = load_vectors()

    ft_set = set(ft_vec.keys())
    cas_set = set(cas_neg['lem'].unique())

    inter_set = set(ft_set & cas_set)

    print("Fast text vocab: %d \nCAS vocab: %d \nIntersection vocab: %d"%(len(ft_set), len(cas_set), len(inter_set)))
elif params['pretrainedEmbed']=='bin':
    from gensim.models.fasttext import load_facebook_vectors
    
    # query words with ft_vec.wv['word']
    ft_vec = load_facebook_vectors('../embeddings/cc.fr.300.bin')
    
else:
    ft_vec = None

# Training

In [ ]:
result_dict = CV_train(
    cas_neg,
    params,
    embeddings=ft_vec,
)    

models = result_dict['models']
model_function = result_dict['model_function']
scores = result_dict['scores']


In [ ]:
score_df = format_results(scores, classes)

score_df

# Saving Experiment

In [ ]:
es = ExpSaver(
    config=params,
    model=models[0],
    model_function=embed_bilstm,
    folder='../logs',
    results=score_df,
)

es.save()

# Get best results (may not be the best way to do so)

In [ ]:
from train.results import get_best_results

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', 57)

get_best_results(n_best=10)